In [1]:
import sys

sys.path.append('/Users/yujunwon/Project/dancify/ai/yujun/complete')           # 경로 설정 필요

import angle_calculation

---

### 고려할 점
1. dancer_joint와 danceable_joint 중에서 하나에 맞춰서 비교해야 함 → 둘 중에서 짧은 것에 맞춰서 일단 구현
2. 프레임 단위로 비교할 때, 앞뒤 5프레임 정도를 평균내서 하는 것은 어떨지에 대해서 생각해보기 → 폐기

In [11]:
result = angle_calculation.calculate_angle_difference("../point_sample/dancer.json", "../point_sample/danceable.json")

In [10]:
len(angle_calculation.calculate_angle_difference("../point_sample/test_data.json", "../point_sample/test_data_2.json"))

# test_data.json과 test_data_2.json은 각각 602, 601의 길이를 갖는다.
# 현재 구현된 함수에서는 의도적으로 프레임 수가 동일하지 않은 경우에서도 동작하도록 구현했다.

601

- 현재 `calculate_angle_difference()`는 두 JSON 중에서 짧은 것에 대응되도록 구현함

---

### 3. `angle_calculation.py`의 `compute_vector_angle` 함수의 boundary condition 검증

In [4]:
import numpy as np

L1 = np.array([1,1]) - np.array([2,1])
L2 = np.array([1,1]) - np.array([0,1])

# 벡터 크기
magnitude1 = np.linalg.norm(L1)
magnitude2 = np.linalg.norm(L2)

# 각도를 radian과 degree로 계산
angle_rad = np.arccos(np.dot(L1, L2) / (magnitude1 * magnitude2))
angle_deg = np.degrees(angle_rad)

print(angle_rad)
print(angle_deg)

3.141592653589793
180.0


In [ ]:
def calculate_angle_difference(dancer_json_path: str, danceable_json_path: str) -> List[List[float]]:
    """
    두 개의 JSON 파일로부터 키포인트 정보를 로드하여 관절 각도 차이를 계산합니다.

    Args:
        dancer_json_path (str): 춤추는 사람의 JSON 파일 경로
        danceable_json_path (str): 춤을 추는 사람의 JSON 파일 경로

    Returns:
        List[List[float]]: 관절 각도의 차이를 담고 있는 리스트

    Raises:
        FileNotFoundError: 입력된 파일 경로가 존재하지 않을 때
        json.JSONDecodeError: JSON 파일의 구조나 내용이 예상과 다를 때
    """
    try:
        dancer = load_keypoints_from_json(dancer_json_path)
        danceable = load_keypoints_from_json(danceable_json_path)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error occurred while loading keypoints: {e}")
        raise

    try:
        dancer_joint = calculate_joint_angles(dancer)
        danceable_joint = calculate_joint_angles(danceable)
    except Exception as e:
        print(f"Error occurred while calculating joint angles: {e}")
        raise

    joint_list = ['left_pelvic_joint', 'right_pelvic_joint', 'left_shoulder_joint', 'right_shoulder_joint',
                  'left_elbow_joint', 'right_elbow_joint', 'left_knee_joint', 'right_knee_joint']

    diff_list = []

    for dancer_joint_item, danceable_joint_item in zip(dancer_joint, danceable_joint):
        diff = [abs(dancer_joint_item[joint] - danceable_joint_item[joint])
                for joint in joint_list]
        diff_list.append(list(diff))

    return diff_list